In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("./subjects/")
import pickle
import matplotlib.pyplot as plt
import itertools
from sklearn.cluster import KMeans
import tensorflow as tf
import time
import random, math
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import  euclidean_distances
import glob
import re
from sklearn.neighbors import KDTree
from Utils_Functions import generate_dataset, eod
from sdv.metadata import SingleTableMetadata
from sdmetrics.single_table import LogisticDetection, SVCDetection
import pandas as pd
from sdv.sampling import Condition

import numpy as np
from sdmetrics.column_pairs import DiscreteKLDivergence as metric
from sdmetrics.single_table import BinaryAdaBoostClassifier, BinaryDecisionTreeClassifier, BinaryLogisticRegression, BinaryMLPClassifier    

import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
from sklearn.preprocessing import StandardScaler,MinMaxScaler,normalize

def generate_dataset(data, graph, edges, sens_index, priv_group, unpriv_group):
    #global  edge,df_new_dic
    dataset_types = [str(data[i].dtype) for i in data.columns]
    succ_generated = 0
    generation_coef = 10
    
    graph_dic ={}
    for i in graph.columns[1:]:
        if np.where(graph[i])[0].shape[0]==0:
            graph_dic[i]=None
        else:
            graph_dic[i]= graph['Unnamed: 0'][np.where(graph[i])[0]].values

    final_df = pd.DataFrame(columns = data.columns) 
    trial = 0
    not_interesting = False
    while final_df.shape[0]<data.shape[0]:

        if trial > 20:
            not_interesting = True
            return None #succ_generated/( trial *  data.shape[0]*generation_coef) 
        df_new_dic ={}
        for edge in graph.sum().index[np.where(graph.sum()==0)[0]]:
            df_new_dic[edge] = np.random.choice(np.unique(data[edge]), size = data.shape[0]*generation_coef) 

        statring_atts = graph.sum().index[np.where(graph.sum()==0)[0]]
        
        while statring_atts.shape[0] != graph['Unnamed: 0'].shape[0]:
            
            for att in graph_dic.keys():
                if att not in statring_atts:
                    if 0 in  [1 if graph_dic[att][i] in statring_atts else 0 for i in range(graph_dic[att].shape[0])]:

                        continue
                    else:
                        edge_logits = 0
                        
                        for cause in graph_dic[att]:
                            edge_logits += (edges[cause+att] * df_new_dic[cause])
                        if np.unique(data[att]).shape[0]==2:
                            df_new_dic[att] =  tfb.distributions.Bernoulli(logits=edge_logits + edges[att+'0'] ).sample().numpy()
                        elif 'float' in dataset_types[np.where(data.columns==att)[0][0]]:

                            df_new_dic[att] =  tfb.distributions.Normal(loc=(edge_logits+ edges[att+'0']), scale= edges['sigma_h']).sample().numpy()
                        
                        else:    
                            df_new_dic[att] =  tfb.distributions.Poisson(rate=tf.exp(edge_logits+ edges[att+'0']) ).sample().numpy()  

                        statring_atts = np.append(statring_atts,att) 

         
        new_df = pd.DataFrame(columns = data.columns)
        for col in new_df.columns:
            new_df[col] = df_new_dic[col].astype(str(data[col].dtype))
        

        final_df = pd.concat([final_df,new_df]).reset_index(drop=True)
        final_df = final_df.drop_duplicates()

        #print(succ_generated,trial)
        trial += 1

    
    min_input ={}
    max_input= {}
    for col in data.columns:
        min_input[col] = data[col].min()
        max_input[col] = data[col].max()
    for col in final_df.columns:
        final_df[col] = final_df[col].apply(lambda x: x if x >=min_input[col] and x <=max_input[col] else None)

    final_df.dropna(inplace=True)
    final_df.reset_index(drop=True, inplace=True)
    for col in final_df.columns:
        final_df[col] = final_df[col].astype(str(data[col].dtype))
#     probs={}
#     for val in data[data.columns[sens_index]].unique():
#         probs[val] = [data[(data[data.columns[sens_index]]==val) & (data['y']==1)].shape[0], data[(data[data.columns[sens_index]]==val) & (data['y']==0)].shape[0]]
#     #print(probs)
#     #for val in df['age'].unique():

#     syn_df = pd.DataFrame(columns = data.columns)
#     for val in probs.keys():

#         syn_df  = pd.concat([syn_df,final_df[(final_df[data.columns[sens_index]]==val) & (final_df['y']==1)].sample(n=probs[val][0])])
#         syn_df  = pd.concat([syn_df,final_df[(final_df[data.columns[sens_index]]==val) & (final_df['y']==0)].sample(n=probs[val][1])])

#     for col in syn_df.columns:
#         syn_df[col] = syn_df[col].astype(str(data[col].dtype))
    return final_df
dataset ='Adult'

if dataset == 'Adult':
    sens_index = 7
    priv_group = 1
    unpriv_group = 0
    data_file_name = 'adult_org-Copy1.csv'
if dataset == 'Compas':
    sens_index = 1
    priv_group = 1
    unpriv_group = 0
    data_file_name = 'compas-Copy1'
if dataset == 'Bank':
    sens_index = 0
    priv_group = 5
    unpriv_group = 3
    data_file_name = 'bank'
if dataset == 'Heart':
    sens_index = 0
    priv_group = 1
    unpriv_group = 0 
    data_file_name = 'heart_processed_1'
if dataset == 'Law':
    sens_index = 1
    priv_group = 1
    unpriv_group = 0
    data_file_name = 'law.csv'

if dataset == 'Student':
    sens_index = 0
    priv_group = 1
    unpriv_group = 0  
    data_file_name = 'students-processed_2'
df = pd.read_csv('./subjects/datasets/'+data_file_name)
df = df.drop_duplicates()
X1 = df.to_numpy()[:,:-1]
Y1 = df.to_numpy()[:,-1]

    
for Algorithm in ['pc','ges','simy']:
    print('Algorithm', Algorithm)
    succ_rate_list =[]
    for edge_list_filename in glob.glob('./'+dataset+'_Analysis/'+Algorithm+'/PP/*.csv'):
        #print(edge_list_filename)
        file_num = int(re.findall(r'\d+', edge_list_filename)[0])
        
    #     if file_num in [5,7,10,12]:
    #         continue

        try:
            graph_filename = './'+dataset+'_Analysis/'+Algorithm+'/DAGs/'+dataset+'_'+Algorithm+'_DAG_{file_num}.csv'.format(file_num=file_num)
            graph = pd.read_csv(graph_filename)
            

            #edge_list_filename = './'+dataset+'_Analysis/'+Algorithm+'/PP/'+dataset+'_'+Algorithm+'_pp_{file_num}.csv'.format(file_num=file_num)
            edges_list = pd.read_csv(edge_list_filename)

            if dataset=='Bank' and Algorithm=='simy':
                graph.columns = [i.replace('1','') for i in graph.columns]
                graph[graph.columns[0]] = [i.replace('1','') for i in graph[graph.columns[0]]]
                edges_list.columns = [i.replace('1','') for i in edges_list.columns]
            edges_list = edges_list[edges_list.columns[1:-1]].mean()

        except:
            print('Not a DAG! ',file_num)
            continue
        
            #ave_dist.append(euclidean_distances(X1[np.where(KMean.labels_==[i])],[KMean.cluster_centers_[i]]).max())
        for i in range(1):

            syn_df = generate_dataset(df, graph, edges_list,sens_index, priv_group, unpriv_group)
            #final_df= final_df.astype('int64')
            input(syn_df.shape[0])
            
            metadata = SingleTableMetadata()
            metadata.detect_from_dataframe(df)
            
            rnd_df = pd.DataFrame(columns = df.columns)
            for col in rnd_df.columns:
                if 'float' in str(df[col].dtype):
                    rnd_df[col]  = np.random.uniform(low=df[col].min(),high=df[col].max(),size=df.shape[0])

                else:
                    rnd_df[col]  = np.random.choice(np.unique(df[col]),size=df.shape[0])


            CTGAN_kl = round(metric.compute(real_data=df, synthetic_data=syn_df),2)
            CTGAN_LD  = round(LogisticDetection.compute(real_data=df,synthetic_data=syn_df,metadata=metadata),2)

            # example of calculating the frechet inception distance


            # calculate frechet inception distance
            def calculate_fid(act1, act2):
             # calculate mean and covariance statistics
                
                se = StandardScaler()
                act1 = se.fit_transform(act1.to_numpy())
                act1 = normalize(act1)
                act2 = se.fit_transform(act2.to_numpy())
                act2 = normalize(act2)
                mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
                mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
                # calculate sum squared difference between means
                ssdiff = numpy.sum((mu1 - mu2)**2.0)
                # calculate sqrt of product between cov
                covmean = sqrtm(sigma1.dot(sigma2))
                # check and correct imaginary numbers from sqrt
                if iscomplexobj(covmean):
                    covmean = covmean.real
                # calculate score
                fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
                return fid

                # define two collections of activations

            df_F1 = BinaryLogisticRegression.compute(test_data=df,train_data=df,target=df.columns[-1],metadata=metadata)
            CTGAN_F1 = BinaryLogisticRegression.compute(test_data=df,train_data=syn_df,target=df.columns[-1],metadata=metadata)
            CTGAN_F1_diff = round(df_F1 - CTGAN_F1,3)
            CTGAN_fid = round(calculate_fid(df, syn_df),3)

            #print('-------------------')
            #print(dataset)
            print(f'&FID {CTGAN_fid} & KL {CTGAN_kl}& LR_D {CTGAN_LD} &  F1_diff {CTGAN_F1_diff}&')

            print('-------------------')

In [190]:
conditioned =[]
probs={}
for val in df[df.columns[sens_index]].unique():
    probs[val] = [df[(df[df.columns[sens_index]]==val) & (df[df.columns[-1]]==1)].shape[0], df[(df[df.columns[sens_index]]==val) & (df[df.columns[-1]]==0)].shape[0]]
#print(probs)
#for val in df['age'].unique():

syn_df = pd.DataFrame(columns = df.columns)
for val in probs.keys():
    syn_df  = pd.concat([syn_df,final_df[(final_df[df.columns[sens_index]]==val) & (final_df[df.columns[-1]]==1)].sample(n=probs[val][0])])
    syn_df  = pd.concat([syn_df,final_df[(final_df[df.columns[sens_index]]==val) & (final_df[df.columns[-1]]==0)].sample(n=probs[val][1])])

for col in syn_df.columns:
    syn_df[col] = syn_df[col].astype(str(df[col].dtype))